<a href="https://colab.research.google.com/github/Factorito/AIportplio/blob/main/Termproject_Task2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# 구글 드라이브 공유 링크로 파일 다운로드
!pip install -U gdown
!pip install -qqq datasets # huggingface's lib.
!pip install -qqq transformers==4.48.3
!pip install -qqq accelerate==0.28.0
!pip install tensorboard
!pip install -U accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.7/9.7 MB 50.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.1/290.1 kB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 64.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 56.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 43.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2

In [ ]:
import gdown
import zipfile
import os

# 파일 ID 입력
file_id = "1m4sgrQ8xTGl17IIABtfeZm89AgGt_SZI"

# 다운로드 받을 파일 이름 지정
url = f"https://drive.google.com/uc?id={file_id}"

output = "train.zip"  # 원하는 이름으로 변경 가능

# 다운로드 수행
gdown.download(url, output, quiet=False)

# 결과 파일 저장 경로
os.makedirs("release", exist_ok=True)

# 압축 해제할 경로
extract_dir = "data"
os.makedirs(extract_dir, exist_ok=True)

# 압축 풀기
with zipfile.ZipFile("train.zip", 'r') as zip_ref:
    zip_ref.extractall(extract_dir)

print("압축 해제 완료:", extract_dir)

Downloading...
From (original): https://drive.google.com/uc?id=1m4sgrQ8xTGl17IIABtfeZm89AgGt_SZI
From (redirected): https://drive.google.com/uc?id=1m4sgrQ8xTGl17IIABtfeZm89AgGt_SZI&confirm=t&uuid=f11c7a43-c733-42e7-9d71-ec39561b5413
To: /content/train.zip
100%|██████████| 3.11G/3.11G [00:42<00:00, 73.3MB/s]


압축 해제 완료: data


In [ ]:
# Task 2 에서 검색할 Top-k 유사 이미지 개수
# Task 2 진행시만 사용
Top_k = 5

#Embedding code(not Test code)


In [ ]:
import os
import torch
import pandas as pd
import numpy as np
from PIL import Image, UnidentifiedImageError
from tqdm.auto import tqdm
from sklearn.metrics.pairwise import cosine_similarity
from torch.utils.data import Dataset, DataLoader
from transformers import ViTModel, ViTImageProcessor

# --------- Config ---------
model_name = "google/vit-base-patch16-224"
batch_size = 32
num_workers = 2
train_csv_path = './data/train/train_labels.csv'
train_img_dir = './data/train/images'

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# --------- Load Model & Processor ---------
model = ViTModel.from_pretrained(model_name).to(device)
processor = ViTImageProcessor.from_pretrained(model_name)

# --------- Dataset Class ---------
class ImageDataset(Dataset):
    def __init__(self, df, img_dir, processor):
        self.df = df
        self.img_dir = img_dir
        self.processor = processor

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        path = os.path.join(self.img_dir, row['file_name'])
        try:
            image = Image.open(path).convert("RGB")
        except (UnidentifiedImageError, FileNotFoundError):
            image = Image.new("RGB", (224, 224), (255, 255, 255))
        image_tensor = self.processor(images=image, return_tensors="pt")['pixel_values'].squeeze(0)
        return image_tensor, row['file_name'], row['style'], row['fruit']

# --------- Embedding Function ---------
def get_embeddings(dataloader):
    embeddings, metadata = [], []
    for batch_imgs, fnames, styles, fruits in tqdm(dataloader):
        inputs = {"pixel_values": batch_imgs.to(device)}
        with torch.no_grad():
            outputs = model(**inputs)
            cls_embeds = outputs.last_hidden_state[:, 0, :].cpu()
        embeddings.append(cls_embeds)
        metadata.extend(zip(fnames, styles, fruits))
    return torch.cat(embeddings, dim=0), metadata

# --------- Preprocessing ---------
def preprocess_image(image_path):
    try:
        image = Image.open(image_path).convert("RGB")
    except (UnidentifiedImageError, FileNotFoundError):
        image = Image.new("RGB", (224, 224), (255, 255, 255))
    return processor(images=image, return_tensors="pt").to(device)

# --------- Single Image Embedding ---------
def get_embedding(inputs):
    with torch.no_grad():
        outputs = model(**inputs)
        return outputs.last_hidden_state[:, 0, :].cpu().squeeze(0)



# --------- Main Pipeline ---------
if __name__ == '__main__':
    # Train Embeddings
    train_df = pd.read_csv(train_csv_path)
    train_dataset = ImageDataset(train_df, train_img_dir, processor)
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=False, num_workers=num_workers)
    print("Embedding train images...")
    train_embeddings, train_metadata = get_embeddings(train_loader)

torch.save(model, "model.pt") # 모델 저장

#임베딩/메타데이터 저장
torch.save({
    "embeddings": train_embeddings,
    "metadata": train_metadata
}, "train_embedding_data.pt")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/69.7k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/346M [00:00<?, ?B/s]

Some weights of ViTModel were not initialized from the model checkpoint at google/vit-base-patch16-224 and are newly initialized: ['vit.pooler.dense.bias', 'vit.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


preprocessor_config.json:   0%|          | 0.00/160 [00:00<?, ?B/s]

Embedding train images...


  0%|          | 0/225 [00:00<?, ?it/s]

TypeError: save() missing 1 required positional argument: 'f'

#TOP K retrieve code

In [ ]:
#데이터 업데이트
import gdown
import zipfile
import os
import torch

# 파일 ID재입력

file_id = "1O4KuLXp9FlxBrkMC_zFnMKtLZ2gMEeT4"
url = f"https://drive.google.com/uc?export=download&id={file_id}"

output = "train_embedding_data.pt"  # 원하는 이름으로 변경 가능

# 다운로드 수행
gdown.download(url, output, quiet=False)

# 데이터 저장 경로
os.makedirs("data", exist_ok=True)

print("모델 다운로드 완료 완료:")
# 데이터 로딩
data = torch.load("train_embedding_data.pt", map_location="cpu")

Downloading...
From: https://drive.google.com/uc?export=download&id=1O4KuLXp9FlxBrkMC_zFnMKtLZ2gMEeT4
To: /content/train_embedding_data.pt
100%|██████████| 23.2M/23.2M [00:00<00:00, 98.8MB/s]


모델 다운로드 완료 완료:


In [ ]:
data #embbedding tensor 확인

{'embeddings': tensor([[ 1.3034,  0.6922,  0.1350,  ..., -0.1215,  0.5286, -0.6625],
         [ 1.1940,  0.0739,  0.1524,  ..., -0.0422,  0.8363, -0.7417],
         [ 1.1219,  1.0568,  1.9525,  ...,  0.8475,  0.0817,  0.2119],
         ...,
         [-0.1704,  0.6194, -0.0912,  ...,  0.1127, -0.3878,  0.1308],
         [-0.1704,  0.6194, -0.0912,  ...,  0.1127, -0.3878,  0.1308],
         [-0.1704,  0.6194, -0.0912,  ...,  0.1127, -0.3878,  0.1308]]),
 'metadata': [('0.jpg', tensor(0), tensor(1)),
  ('1.jpg', tensor(0), tensor(1)),
  ('2.jpg', tensor(0), tensor(1)),
  ('3.jpg', tensor(0), tensor(1)),
  ('4.jpg', tensor(0), tensor(1)),
  ('5.jpg', tensor(0), tensor(1)),
  ('6.jpg', tensor(0), tensor(1)),
  ('7.jpg', tensor(0), tensor(1)),
  ('8.jpg', tensor(0), tensor(1)),
  ('9.jpg', tensor(0), tensor(1)),
  ('10.jpg', tensor(0), tensor(1)),
  ('11.jpg', tensor(0), tensor(1)),
  ('12.jpg', tensor(0), tensor(1)),
  ('13.jpg', tensor(0), tensor(1)),
  ('14.jpg', tensor(0), tensor(1)),
  

In [ ]:
# --------- Top-K Retrieval ---------
def retrieve_topk(test_paths, train_embeddings, train_metadata, k):
    results = []
    for path in tqdm(test_paths):
        inputs = preprocess_image(path)
        test_embedding = get_embedding(inputs)
        sims = cosine_similarity(test_embedding.reshape(1, -1), train_embeddings.numpy()).flatten()
        topk_idxs = sims.argsort()[::-1][:k]

        topk_tuples = [
            (int(train_metadata[i][0].split('.')[0]),  # id (remove .jpg)
             int(train_metadata[i][1]),                 # style
             int(train_metadata[i][2]))                 # fruit
            for i in topk_idxs
        ]
        results.append(topk_tuples)
    return results

# --------- Save Results ---------
def save_structured_results(results, output_path):
    with open(output_path, "w") as f:
        for topk_list in results:
            f.write(f"{topk_list}\n")


# --------- Embedding Function ---------
def get_embeddings(dataloader):
    embeddings, metadata = [], []
    for batch_imgs, fnames, styles, fruits in tqdm(dataloader):
        inputs = {"pixel_values": batch_imgs.to(device)}
        with torch.no_grad():
            outputs = model(**inputs)
            cls_embeds = outputs.last_hidden_state[:, 0, :].cpu()
        embeddings.append(cls_embeds)
        metadata.extend(zip(fnames, styles, fruits))
    return torch.cat(embeddings, dim=0), metadata

# --------- Preprocessing ---------
def preprocess_image(image_path):
    try:
        image = Image.open(image_path).convert("RGB")
    except (UnidentifiedImageError, FileNotFoundError):
        image = Image.new("RGB", (224, 224), (255, 255, 255))
    return processor(images=image, return_tensors="pt").to(device)

# --------- Single Image Embedding ---------
def get_embedding(inputs):
    with torch.no_grad():
        outputs = model(**inputs)
        return outputs.last_hidden_state[:, 0, :].cpu().squeeze(0)


In [ ]:
import os
import torch
import pandas as pd
import numpy as np
from PIL import Image, UnidentifiedImageError
from tqdm.auto import tqdm
from sklearn.metrics.pairwise import cosine_similarity
from torch.utils.data import Dataset, DataLoader
from transformers import ViTModel, ViTImageProcessor

processor = ViTImageProcessor.from_pretrained("google/vit-base-patch16-224")

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = ViTModel.from_pretrained("google/vit-base-patch16-224").to(device)

data = torch.load("train_embedding_data.pt", map_location="cpu")

test_embeddings = data["embeddings"]

test_metadata = data["metadata"]


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


preprocessor_config.json:   0%|          | 0.00/160 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/69.7k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/346M [00:00<?, ?B/s]

Some weights of ViTModel were not initialized from the model checkpoint at google/vit-base-patch16-224 and are newly initialized: ['vit.pooler.dense.bias', 'vit.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:

# Test 이미지 목록
test_img_dir = './data/test/images' #coustom plz
output_txt = 'release/test.task2.txt'

test_paths = sorted([
        os.path.join(test_img_dir, fname)
        for fname in os.listdir(test_img_dir)
        if fname.lower().endswith((".jpg", ".jpeg", ".png"))
    ])

print("Retrieving top-k results...")
results = retrieve_topk(test_paths, test_embeddings, test_metadata, Top_k)

save_structured_results(results, output_txt)
print(f"Saved Top-{Top_k} results to {output_txt}")

Retrieving top-k results...


  0%|          | 0/7200 [00:00<?, ?it/s]

KeyboardInterrupt: 